In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from pymongo import MongoClient

In [44]:
pd.set_option('display.max_rows', 500)

In [3]:
mongo_connection = "INSERT_URL"

In [7]:
MULT_COUNTY_JAIL_BOOKINGS_URL = "http://www.mcso.us/PAID/Home/SearchResults"

In [4]:
client = MongoClient(mongo_connection)

In [18]:
db = client.data

In [48]:
db.charges.drop()

In [12]:
CSV_url = "csvs/inmate_details.csv"

In [13]:
INMATE_COLUMNS = ['SWIS ID', 'Name', 'Age', 'Gender', 'Race', 'Height', 'Weight', 'Hair', 'Eyes', 'Arresting Agency', 'Booking Date', 'Assigned Facility', 'Projected Release Date']

In [49]:
response = requests.post(MULT_COUNTY_JAIL_BOOKINGS_URL)

In [50]:
soup = BeautifulSoup(response.text, 'html.parser')

In [51]:
table = soup.find('table')
table_input = table.prettify()

In [52]:
# parse dates of column at index 1
table_to_df = pd.read_html(table_input, parse_dates=[1])[0]

# sort by date descending
table_to_df = table_to_df.sort_values(by=["Booking Date", "Name"], ascending = [False, True])

In [53]:
links = table.find_all('a')

In [ ]:
for link in links:
    booking_url = link.get('href')
    inmate_link = "http://www.mcso.us" + booking_url
    
    # go to inmate's page
    inmate_response = requests.get(inmate_link)
    soup = BeautifulSoup(inmate_response.text, 'html.parser')
    
    # get table and just grab name
    table_input = soup.find('table').prettify()
    booking_table = pd.read_html(table_input)[0][1]
    
    
    charge_infos = soup.find(id='charge-info')
    charge_items = []
    charges = charge_infos.select('div > div > ol')
    for charge in charges:
        for el in charge.find_all('li'):
            charge_type = el.find('span').getText()
            charge_bail = el.find(class_='charge-bail-display').text
            charge_status = el.find(class_='charge-status-value').text
            charge_item = {
                'Type': charge_type,
                'Bail': charge_bail,
                'Status': charge_status,
                'Name': booking_table[1]
            }
            db.charges.insert_one(charge_item)
    

In [26]:
db.charges.count_documents({})

9525

In [27]:
docs = db.charges.find({})

In [30]:
inmates_charges = [doc for doc in docs]

In [31]:
inmates_charges_df = pd.DataFrame(inmates_charges)

In [45]:
inmates_charges_df

,_id,Type,Bail,Status,Name
0,631d8aea7dba1a69bec4bc4a,UNLAW USE WEAPON (C Felony),Bail: $0,Unsentenced,/PAID/Home/Booking/1545155
1,631d8aea7dba1a69bec4bc4b,FELON POSS FA (C Felony),Bail: $500,Unsentenced,/PAID/Home/Booking/1545155
2,631d8aea7dba1a69bec4bc4c,ATT ELUDE BY VEHCLE (C Felony),Bail: $0,Unsentenced,/PAID/Home/Booking/1545155
3,631d8aea7dba1a69bec4bc4d,ATT ELUDE ON FOOT (A Misdemeanor),Bail: $0,Unsentenced,/PAID/Home/Booking/1545155
4,631d8aea7dba1a69bec4bc4e,DWS/REV MISD (A Misdemeanor),Bail: $0,Unsentenced,/PAID/Home/Booking/1545155
...,...,...,...,...,...
9519,631d8d3e7dba1a69bec4e179,HARASSMENT - B MISD (B Misdemeanor),Bail: $0,Unsentenced,/PAID/Home/Booking/1537541
9520,631d8d3e7dba1a69bec4e17a,RESIST ARREST (A Misdemeanor),"Bail: $2,000",Unsentenced,/PAID/Home/Booking/1537541
9521,631d8d3e7dba1a69bec4e17b,INTERFER PEACE OFF (A Misdemeanor),Bail: $0,Unsentenced,/PAID/Home/Booking/1537541
9522,631d8d3e7dba1a69bec4e17c,THEFT III (C Misdemeanor),Bail: $0,Unsentenced,/PAID/Home/Booking/1537541


In [38]:
inmates_charges_df['Type'].value_counts()

UNLAW USE WEAPON (C Felony)        471
UUMV (C Felony)                    383
COUNTY HOLD (U Unknown)            355
IDENTITY THEFT (C Felony)          275
FELON POSS FA (C Felony)           266
                                  ... 
UNL DIS INTIMATE IMG (C Felony)      1
CRIM MISCH I - DV (C Felony)         1
TRESPASS II (A Misdemeanor)          1
ROBBERY I DV (A Felony)              1
ATT THEFT II (B Misdemeanor)         1
Name: Type, Length: 293, dtype: int64